In [2]:
# -*- coding: utf-8 -*-
import numpy as np
import cv2
import os
import pandas as pd
import h5py

def get_name(index, hdf5_data):
    name = hdf5_data['/digitStruct/name']
    return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

def get_bbox(index, hdf5_data):
    attrs = {}
    item = hdf5_data['digitStruct']['bbox'][index].item()
    for key in ['label', 'left', 'top', 'width', 'height']:
        attr = hdf5_data[item][key]
        values = [hdf5_data[attr.value[i].item()].value[0][0]
                  for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
        attrs[key] = values
    return attrs
def img_boundingbox_data_constructor(mat_file):
    f = h5py.File(mat_file,'r') 
    all_rows = []
    print('image bounding box data construction starting...')
    bbox_df = pd.DataFrame([],columns=['height','img_name','label','left','top','width'])
    for j in range(f['/digitStruct/bbox'].shape[0]):
        img_name_ = get_name(j, f)
        row_dict = get_bbox(j, f)
        row_dict['img_name'] = img_name_
        all_rows.append(row_dict)
        bbox_df = pd.concat([bbox_df,pd.DataFrame.from_dict(row_dict,orient = 'columns')])
    bbox_df['bottom'] = bbox_df['top']+bbox_df['height']
    bbox_df['right'] = bbox_df['left']+bbox_df['width']
    print('finished image bounding box data construction...')
    return bbox_df

df = img_boundingbox_data_constructor('./digitStruct.mat')

image bounding box data construction starting...
finished image bounding box data construction...


In [13]:
# 26722
# 6680
count = 0
write_line = ''
img_path = ''
for row in df.iterrows():
    img_path = ''
    if int(row[1]['img_name'].split('.')[0]) <= 26722:
        img_path = './svhn/images/train/' + row[1]['img_name']
        output_path = './svhn/labels/train/' + row[1]['img_name'].split('.')[0] + '.txt'
    else:
        img_path = './svhn/images/val/' + row[1]['img_name']
        output_path = './svhn/labels/val/' + row[1]['img_name'].split('.')[0] + '.txt'
    
    img = cv2.imread(img_path)
#     print(img.shape)
    img_width = img.shape[1]
    img_height = img.shape[0]
    
    if row[0] != 0:
        write_line += '\n'
    if ((row[1]['left']+row[1]['right'])/2)/img_width > 1:
        print('1: ', ((row[1]['left']+row[1]['right'])/2)/img_width)
        print((row[1]['left']+row[1]['right'])/2)
        print('img_width:', img_width)
        print('img_height: ', img_height)
        print("error")
        print('img_path', img_path)
        print('name: ', row[1]['img_name'])
        break;
    if ((row[1]['top']+row[1]['bottom'])/2)/img_height > 1:
        print('2: ', ((row[1]['top']+row[1]['bottom'])/2)/img_height)
        print((row[1]['top']+row[1]['bottom'])/2)
        print('img_width:', img_width)
        print('img_height: ', img_height)
        print("error")
        print('img_path', img_path)
        print('name: ', row[1]['img_name'])
        break;
    if row[1]['width']/img_width > 1:
        print('3: ',row[1]['width']/img_width)
        print(row[1]['width'])
        print('img_width:', img_width)
        print('img_height: ', img_height)
        print("error")
        print('img_path', img_path)
        print('name: ', row[1]['img_name'])
        break;
    if row[1]['height']/img_height > 1 and row[1]['img_name'] != '12668.png':
        print('4: ', row[1]['height']/img_height)
        print(row[1]['height'])
        print('img_width:', img_width)
        print('img_height: ', img_height)
        print("error")
        print('img_path', img_path)
        print('name: ', row[1]['img_name'])
        break;
    write_line += str(int(row[1]['label']-1)) + ' '
    write_line += str(((row[1]['left']+row[1]['right'])/2)/img_width) + ' '
    write_line += str(((row[1]['top']+row[1]['bottom'])/2)/img_height) + ' '
    write_line += str(row[1]['width']/img_width) + ' '
    if row[1]['img_name'] == '12668.png':
        write_line += str(img_height/img_height)
    else:
        write_line += str(row[1]['height']/img_height)

    with open(output_path, 'a') as output_file:
        output_file.write(write_line)
    write_line = ''
    count += 1

print(count+1)

73258
